In [110]:
import json 
import pandas as pd
import requests

# Read API key credentials
api = pd.read_json('~/Coding/APIs/twitter_keys.json')['DSAN_501-project']

# Save credentials
consumer_key=api.loc["key"]
consumer_secret=api.loc["key_secret"]
access_token=api.loc["access_token"]
access_token_secret=api.loc["access_token_secret"]
bearer_token=api.loc["bearer_token"]

# Read in dataset
incidents = pd.read_csv('../../../../data/02-clean-data/SSDB/incident.csv')
incidents['Date'] = pd.to_datetime(incidents['Date'])

# Filter out incidents that occurred before 2006-03-21 (day of first ever tweet)
incidents_twitter = incidents.copy()[incidents['Date'] > pd.to_datetime('2006-03-21 15:50:00')]

# Add new data column marking one week after the incident
incidents_twitter['Date_Week_Later'] = incidents_twitter['Date'] + pd.Timedelta(days=7)

# Convert date columns to YYYY-MM-DDTHH:mm:ssZ (ISO 8601/RFC 3339) for API requests
incidents_twitter['Query_Date'] = incidents_twitter['Date'].dt.strftime('%Y-%m-%d') + "T00:00:00Z"
incidents_twitter['Query_Date_Week_Later'] = incidents_twitter['Date_Week_Later'].dt.strftime('%Y-%m-%d') + "T00:00:00Z"

# Construct get request parameters, save IDs to list
ids = incidents_twitter['Incident_ID']
queries = "-is:retweet ((school (shooting OR shootings)) OR (" + incidents['City'] + " (shooting OR shootings)) OR (" + incidents['School'] + "))"
start_times = incidents_twitter['Query_Date']
end_times = incidents_twitter['Query_Date_Week_Later']

# Specify relative filepath for outputs
rel_filepath = "../../../../data/00-raw-data/Twitter/"

# Define function for retrieving twitter data (adapted from Lab-1.2)
def search_twitter(query, start_time, end_time, bearer = bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/all?query={}&start_time={}&end_time={}&tweet.fields=text,author_id,created_at,geo,lang".format(query, start_time, end_time)
    print("--------------",url,"--------------")
    response = requests.request("GET", url, headers=headers)
    #print(response.status_code)
    # print(response.text)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [112]:
queries[0]

'-is:retweet ((school (shooting OR shootings)) OR (Chicago (shooting OR shootings)) OR (Gresham School of Excellence))'

In [81]:
json_response = search_twitter(query=queries[300], start_time=start_times[300], end_time=end_times[300])

output = json.dumps(json_response, indent=4, sort_keys=True)

with open(rel_filepath + ids[300] + '.json', 'w') as outfile:
    outfile.write(output)

In [111]:
for i, id in enumerate(ids):
    print(i)
    print(id)

0
20220620ILGRC
1
20220613WAMAE
2
20220610ALBYB
3
20220609ALWAG
4
20220608ARLIL
5
20220607MIPED
6
20220605INWEG
7
20220601CAULL
8
20220531LAMON
9
20220530CAHEL
10
20220529ILDAC
11
20220526TXDUA
12
20220526SCMEG
13
20220525ILSTC
14
20220524WIRIM
15
20220524TXROU
16
20220524DCPOW
17
20220523PASIP
18
20220520VAPOD
19
20220520TNEAC
20
20220520OHCAC
21
20220520ILSOP
22
20220520ALMAT
23
20220519VAGER
24
20220519MIEAK
25
20220519LAHAH
26
20220518TNRIM
27
20220518FLPAP
28
20220517ILWAC
29
20220517CASAS
30
20220516TXMEM
31
20220515NHBEB
32
20220515ILMEP
33
20220513GASOM
34
20220513FLALW
35
20220512TXHEH
36
20220512ARHOH
37
20220511FLJAJ
38
20220509NYEDS
39
20220509GARIS
40
20220505OHLOL
41
20220505ALDOD
42
20220503CAARS
43
20220501VALOM
44
20220501OHHAC
45
20220430PAMCJ
46
20220427TXMOS
47
20220427INCOS
48
20220426MIASI
49
20220426GASOM
50
20220425WIWIM
51
20220425GAMAM
52
20220424MOHAF
53
20220422ORHOS
54
20220422DCEDW
55
20220422CAMOR
56
20220421NDMOM
57
20220416IAMED
58
20220415VAGAW
59
2022

-------------- https://api.twitter.com/2/tweets/search/all?query=-is:retweet ((school (shooting OR shootings)) OR (Hauppauge (shooting OR shootings)) OR (Hauppauge Middle School))&start_time=2021-08-26T00:00:00Z&end_time=2021-09-02T00:00:00Z&tweet.fields=text,author_id,created_at,geo,lang --------------
{
    "data": [
        {
            "author_id": "1472502049",
            "created_at": "2021-09-01T23:59:42.000Z",
            "id": "1433218036392239104",
            "lang": "en",
            "text": "Manhunt underway for suspect after student is killed in shooting at North Carolina high school - CNN https://t.co/eaxLj3L3K8"
        },
        {
            "author_id": "73801315",
            "created_at": "2021-09-01T23:59:41.000Z",
            "id": "1433218034492231682",
            "lang": "en",
            "text": "Normal start of the school year \ud83e\udd26\u200d\u2640\ufe0f. Why anyone thinks shooting at a school is okay is beyond me. America has gone mad!!!!!!"
        }